In [14]:
import pandas as pd 
import json 
import os

In [15]:
def create_mapping_item(item_id):
    item_id = sorted(set(item_id))
    item_id_mapping = dict()
    for i in range(len(item_id)):
        item_id_mapping[item_id[i]] = i
    return item_id_mapping

In [16]:
def json_format_data(data_set):
    json_data = {}
    key = data_set.keys()
    for i in range(data_set.shape[0]):
        row = data_set.iloc[i]
        user, bundle = row[key[0]], row[key[1]]
        if user not in json_data:
            json_data[user] = []
        json_data[user].append(bundle)
    return json_data

In [17]:
def print_txt(data_set, path):
    if os.path.exists(path):
        os.remove(path)
    with open(path, 'a') as f:
        for first_e in data_set:
            for second_e in data_set[first_e]:
                f.write(f'{first_e}\t{second_e}\n')

In [19]:
def split_leave_one_out(dataset_name):
    user_bundle_path = os.path.join(dataset_name, "user_bundle.csv")
    user_item_path = os.path.join(dataset_name, "user_item.csv")
    bundle_item_path = os.path.join(dataset_name, "bundle_item.csv")
    user_bundle = pd.read_csv(user_bundle_path)
    user_item = pd.read_csv(user_item_path)
    bundle_item = pd.read_csv(bundle_item_path)

    item_id = []
    for i in range(bundle_item.shape[0]):
        item_id.append(bundle_item.iloc[i]['item ID'])
    mapping_item_id = create_mapping_item(item_id)
    # print(mapping_item_id)
    # print(len(mapping_item_id))

    bundle_item['item ID'] = bundle_item['item ID'].map(mapping_item_id)
    user_item['item ID'] = user_item['item ID'].map(mapping_item_id)

    user_bundle = user_bundle.drop(columns=['timestamp'])
    ser_item = user_item.drop(columns=['timestamp'])

    train_set = {}
    test_set = {}
    validation_set = {}
    user_bundle_json = json_format_data(user_bundle)
    for user in user_bundle_json:
        if len(user_bundle_json[user]) < 3:
            train_set[user] = user_bundle_json[user]
        else:
            train_set[user] = user_bundle_json[user][:-2]
            validation_set[user] = [user_bundle_json[user][-2]]
            test_set[user] = [user_bundle_json[user][-1]]
    print(len(train_set)) # user for train
    print(len(validation_set)) # user for validation
    print(len(test_set)) # user for test

    user_item = json_format_data(user_item)
    bundle_item = json_format_data(bundle_item)

    # path for output 
    bundle_item_output_path = os.path.join(dataset_name, "bundle_item.txt")
    user_item_output_path = os.path.join(dataset_name, "user_item.txt")
    user_bundle_item_train = os.path.join(dataset_name, "user_bundle_train.txt")
    user_bundle_item_test = os.path.join(dataset_name, "user_bundle_test.txt")
    user_bundle_item_tune = os.path.join(dataset_name, "user_bundle_tune.txt")

    print_txt(train_set, user_bundle_item_train)
    print_txt(validation_set, user_bundle_item_tune)
    print_txt(test_set, user_bundle_item_test)
    print_txt(user_item, user_item_output_path)
    print_txt(bundle_item, bundle_item_output_path)

dataset_name = ['food', 'clothing']
for dataset in dataset_name:
    split_leave_one_out(dataset)

879
218
218
965
243
243
